In [2]:
!ls

optimizers.ipynb  results.ipynb


In [4]:
%load_ext autoreload
%autoreload 2

In [155]:
# from schedulefree import ScheduleFreeWrapper
from mechanic_pytorch import mechanize
# from schedulefree import ScheduleFreeWrapper
import torch
from torch import nn
import torch.optim as optim
from lion_pytorch import Lion

from prodigyopt import Prodigy

from custom_optimizers import signSGD, FrankWolfe, MirrorGD


from torch import functional as F
# from deep import MLP

In [156]:


def lmo_1(x, R):
    y = x.view(-1)
    max_idx = torch.argmax(torch.abs(y))
    mask_flat = torch.zeros_like(y)
    mask_flat[max_idx] = 1
    return -R*mask_flat.view(x.shape)

def lmo_2(x, R):
    return - R * x / x.norm(p=2)

def mirror_entropy_simplex( x, g, R ):
    value = torch.exp(-g) * x 
    return R * value / value.sum()



In [194]:
model = nn.Linear(10,1)

# model.weight.data = model.weight.data ** 2

x = torch.randn(10)  # Входные данные
y = torch.tensor(4.0)  # Целевое значение

# frankwolfe_optimizer_1 = FrankWolfe(params=model.parameters() , lmo =lmo_1)
# frankwolfe_optimizer_2 = FrankWolfe(params=model.parameters() , lmo =lmo_2)
# adamw_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-1, weight_decay = 1e-6 )
mirrorgd_optimizer = MirrorGD(params=model.parameters(),mirror=mirror_entropy_simplex, lr = 1e-2, scale = 2)

num_epochs = 1000
# optimizer = adamw_optimizer
# optimizer = frankwolfe_optimizer_1
# optimizer = frankwolfe_optimizer_2
optimizer = mirrorgd_optimizer
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Обнуляем градиенты

    output = model(x)

    loss = (output - y) ** 2

    # print('Loss :',loss)

    loss.backward()
    optimizer.step()

Group R: [tensor(3.5178), tensor(0.1916)]


In [195]:
model.weight.data

tensor([[1.3789, 0.0000, 0.0000, 0.3087, 0.0000, 1.8302, 0.0000, 0.0000, 0.0000,
         0.0000]])

In [196]:
model.weight.data.norm(p=1) , model.bias.data.norm(p=1)

(tensor(3.5178), tensor(0.1916))

In [197]:
model(x)

tensor([4.0000], grad_fn=<AddBackward0>)

## All optimizers

In [65]:
# Optimizers
adamw_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay = 1e-6 )
sgd_optimizer = optim.SGD(model.parameters(), lr=1e-5, momentum=0.9)
lion_optimizer = Lion(model.parameters(), lr=1e-5, weight_decay=1e-2)
prodigy_optimizer = Prodigy(model.parameters(), lr=1e-5, weight_decay=1e-2)
rmsprob_optimizer = optim.RMSprop(model.parameters(), lr=1e-5, alpha=0.99)
signsgd_optimizer = signSGD(model.parameters(), lr=1e-5)
adagrad_optimizer = optim.Adagrad(model.parameters(), lr=1e-5)
lbfgs_optimizer = torch.optim.LBFGS(model.parameters(), lr=1e-5 , history_size= 40, line_search_fn=True)


# Wrappers
schedulefree_optimizer = ScheduleFreeWrapper( adamw_optimizer, momentum=0.9, weight_decay_at_y=0.1)
schedulefree_optimizer.train()
mechanize_optimizer = mechanize(torch.optim.AdamW, s_decay=0.0, betas=(0.999,0.999999), store_delta=False)(model.parameters(), lr=1e-5, weight_decay = 0)


Using decoupled weight decay


In [7]:
model(torch.Tensor(10))

tensor([-1.4758e+25], grad_fn=<AddBackward0>)

In [11]:
def default_zero_weight_decay_condition(
    module_name: str, module: nn.Module, parameter_name: str, parameter
):
    del module_name, parameter
    return parameter_name.endswith('bias') or isinstance(
        module,
        (
            nn.BatchNorm1d,
            nn.LayerNorm,
            nn.InstanceNorm1d,
        ),
    )

def make_parameter_groups(
    model: nn.Module,
    custom_groups: dict[tuple[str], dict],  # [(fullnames, options), ...]
    zero_weight_decay_condition= default_zero_weight_decay_condition ,
):
    custom_fullnames = set()
    custom_fullnames.update(*custom_groups)
    assert sum(map(len, custom_groups)) == len(
        custom_fullnames
    ), 'Custom parameter groups must not intersect'

    parameters_info = {}  # fullname -> (parameter, needs_wd)
    for module_name, module in model.named_modules():
        for name, parameter in module.named_parameters():
            fullname = f'{module_name}.{name}' if module_name else name
            parameters_info.setdefault(fullname, (parameter, []))[1].append(
                not zero_weight_decay_condition(module_name, module, name, parameter)
            )
    parameters_info = {k: (v[0], all(v[1])) for k, v in parameters_info.items()}

    params_with_wd = {'params': []}
    params_without_wd = {'params': [], 'weight_decay': 0.0}
    custom_params = {k: {'params': []} | v for k, v in custom_groups.items()}

    for fullname, (parameter, needs_wd) in parameters_info.items():
        for fullnames, group in custom_params.items():
            if fullname in fullnames:
                custom_fullnames.remove(fullname)
                group['params'].append(parameter)
                break
        else:
            (params_with_wd if needs_wd else params_with_wd)['params'].append(parameter)
    assert (
        not custom_fullnames
    ), f'Some of the custom parameters were not found in the model: {custom_fullnames}'
    return [params_with_wd, params_without_wd] + list(custom_params.values())

In [12]:
parameter_groups = make_parameter_groups(model,{})


In [14]:
Optimizer = getattr(optim, 'SGD')

In [17]:
Optimizer

torch.optim.sgd.SGD

In [18]:
base_optimizer = torch.optim.RMSprop( parameter_groups, lr=0.0025)

In [44]:
optimizer = ScheduleFreeWrapper( base_optimizer, momentum=0.9, weight_decay_at_y=0.1)

In [42]:
optimizer.train()

In [45]:
optimizer.step()

Exception: Please insert .train() and .eval() calls for the optimizer. See documentation for details

In [ ]:
optim.AdamW(lr = 1e-05)

In [36]:
optimizer = mechanize(torch.optim.AdamW, s_decay=0.0, betas=(0.999,0.999999), store_delta=False)(model.parameters(), lr=1e-5, weight_decay = 0)

In [ ]:
type = "AdamW"
lr = [
    "_tune_",
    "loguniform",
    1e-05,
    0.001,
]
weight_decay = [
    "_tune_",
    "?loguniform",
    0.0,
    1e-06,
    0.0001,
]

In [32]:
optimizer = mechanize(torch.optim.SGD, s_decay=0.0, betas=(0.999,0.999999), store_delta=False)(model.parameters(), lr=0.01)

In [1]:
import os
import sys
_project_dir = os.path.dirname(os.getcwd())
os.environ['PROJECT_DIR'] = _project_dir
sys.path.append(_project_dir)
del _project_dir

from copy import deepcopy
from pathlib import Path
from typing import Any, Optional, Union, cast

import pandas as pd

import lib; lib.configure_libraries()

pd.set_option("display.max_rows", 1000)

In [8]:
DATASET_PROPERTIES = ["task_type", "size", "n_features"]
_DATASETS_INFO: dict[Path, dict[str, Any]] = {}

DATASETS_MAIN = [
    # 'churn',
    # 'california',
    # 'house',
    'adult',
    # 'diamond',
    # 'otto',
    # 'higgs-small',
    # 'black-friday',
    # 'weather-small',
    # 'covtype',
    # 'microsoft',
]
# The datasets from the paper "Why do tree-based models still outperform deep learning on tabular data?"
DATASETS_WHY = [
    # 'classif-cat-large-0-covertype',
    # 'classif-cat-large-0-road-safety',
    # 'classif-cat-medium-0-KDDCup09_upselling',
    # 'classif-cat-medium-1-KDDCup09_upselling',
    # 'classif-cat-medium-2-KDDCup09_upselling',
    # 'classif-cat-medium-0-compass',
    # 'classif-cat-medium-1-compass',
    # 'classif-cat-medium-0-electricity',
    # 'classif-cat-medium-0-rl',
    # 'classif-cat-medium-1-rl',
    # 'classif-cat-medium-2-rl',
    # 'classif-num-large-0-Higgs',
    # 'classif-num-large-0-MiniBooNE',
    # 'classif-num-large-0-jannis',
    # 'classif-num-medium-0-MagicTelescope',
    # 'classif-num-medium-1-MagicTelescope',
    # 'classif-num-medium-2-MagicTelescope',
    # 'classif-num-medium-0-bank-marketing',
    # 'classif-num-medium-1-bank-marketing',
    # 'classif-num-medium-2-bank-marketing',
    # 'regression-num-medium-0-california',
    # 'classif-num-medium-0-credit',
    # 'classif-num-medium-1-credit',
    # 'regression-num-medium-0-house_16H',
    # 'classif-num-medium-0-kdd_ipums_la_97-small',
    # 'classif-num-medium-1-kdd_ipums_la_97-small',
    # 'classif-num-medium-2-kdd_ipums_la_97-small',
    # 'classif-num-medium-0-phoneme',
    # 'classif-num-medium-1-phoneme',
    # 'classif-num-medium-2-phoneme',
    # 'classif-num-medium-3-phoneme',
    # 'classif-num-medium-4-phoneme',
    # 'regression-num-medium-0-pol',
    # 'regression-num-medium-1-pol',
    # 'classif-num-medium-0-wine',
    # 'classif-num-medium-1-wine',
    # 'classif-num-medium-2-wine',
    # 'classif-num-medium-3-wine',
    # 'classif-num-medium-4-wine',
    # 'regression-cat-large-0-SGEMM_GPU_kernel_performance',
    # 'regression-cat-large-0-black_friday',
    # 'regression-cat-large-0-diamonds',
    # 'regression-cat-large-0-nyc-taxi-green-dec-2016',
    # 'regression-cat-large-0-particulate-matter-ukair-2017',
    # 'regression-cat-medium-0-Bike_Sharing_Demand',
    # 'regression-cat-medium-1-Bike_Sharing_Demand',
    # 'regression-cat-medium-0-Brazilian_houses',
    # 'regression-cat-medium-1-Brazilian_houses',
    # 'regression-cat-medium-2-Brazilian_houses',
    # 'regression-cat-medium-0-Mercedes_Benz_Greener_Manufacturing',
    # 'regression-cat-medium-1-Mercedes_Benz_Greener_Manufacturing',
    # 'regression-cat-medium-2-Mercedes_Benz_Greener_Manufacturing',
    # 'regression-cat-medium-3-Mercedes_Benz_Greener_Manufacturing',
    # 'regression-cat-medium-4-Mercedes_Benz_Greener_Manufacturing',
    # 'regression-cat-medium-0-OnlineNewsPopularity',
    # 'regression-cat-medium-0-analcatdata_supreme',
    # 'regression-cat-medium-1-analcatdata_supreme',
    # 'regression-cat-medium-2-analcatdata_supreme',
    # 'regression-cat-medium-3-analcatdata_supreme',
    # 'regression-cat-medium-4-analcatdata_supreme',
    # 'regression-cat-medium-0-house_sales',
    # 'regression-cat-medium-0-visualizing_soil',
    # 'regression-cat-medium-1-visualizing_soil',
    # 'regression-cat-medium-2-visualizing_soil',
    # 'regression-cat-medium-0-yprop_4_1',
    # 'regression-cat-medium-1-yprop_4_1',
    # 'regression-cat-medium-2-yprop_4_1',
    # 'regression-num-large-0-year',
    # 'regression-num-medium-0-Ailerons',
    # 'regression-num-medium-1-Ailerons',
    # 'regression-num-medium-2-Ailerons',
    # 'regression-num-medium-0-MiamiHousing2016',
    # 'regression-num-medium-1-MiamiHousing2016',
    # 'regression-num-medium-2-MiamiHousing2016',
    # 'regression-num-medium-0-cpu_act',
    # 'regression-num-medium-1-cpu_act',
    # 'regression-num-medium-2-cpu_act',
    # 'regression-num-medium-0-elevators',
    # 'regression-num-medium-1-elevators',
    # 'regression-num-medium-0-fifa',
    # 'regression-num-medium-1-fifa',
    # 'regression-num-medium-0-houses',
    # 'regression-num-medium-0-isolet',
    # 'regression-num-medium-1-isolet',
    # 'regression-num-medium-2-isolet',
    # 'regression-num-medium-0-medical_charges',
    # 'regression-num-medium-0-sulfur',
    # 'regression-num-medium-1-sulfur',
    # 'regression-num-medium-2-sulfur',
    # 'regression-num-medium-0-superconduct',
    # 'regression-num-medium-0-wine_quality',
    # 'regression-num-medium-1-wine_quality',
    # 'regression-num-medium-2-wine_quality',
]
DATASETS_ALL = DATASETS_MAIN + ['weather-big'] + DATASETS_WHY


def get_dataset_info(dpath: Union[str, Path]) -> dict:
    dpath = lib.get_path(dpath)
    if dpath in _DATASETS_INFO:
        return _DATASETS_INFO[dpath]

    dataset = lib.Dataset.from_dir(dpath, None)
    _DATASETS_INFO[dpath] = {
        'dataset': (
            dpath.name.upper()[:2] if dpath.parent == lib.DATA_DIR and dpath.name in DATASETS_MAIN
            else 'WE (full)' if dpath.parent == lib.DATA_DIR and dpath.name == 'weather-big'
            else dpath.name
        ),
        'task_type': dataset.task_type.value,
        'size': dataset.size(None),
        'n_features': dataset.n_features,   
    }
    return deepcopy(_DATASETS_INFO[dpath])


def load_record(output: Union[str, Path]):
    output = lib.get_path(output)
    report = lib.load_report(output)
    if lib.EXP_DIR in output.parents and '/exp/npt/' in str(output):
        # The NPT reports do not follow the required format,
        # so we infer the dataset path from the output path.
        dpath = ':data/' + list(output.relative_to(lib.EXP_DIR / 'npt').parents)[-2].name
    else:
        if report["function"] == 'bin.tune.main':
            report = report["best"]

        if report["function"] == 'bin.ensemble.main':
            dpath = report["data"]
        else:
            data = report["config"]["data"]
            dpath = data if isinstance(data, str) else data['path']
            del data

    record = get_dataset_info(dpath)
    for part in lib.Part:
        if part.value in report["metrics"]:
            score = report["metrics"][part.value]["score"]
            if record['dataset'] == 'HO':
                # Prettify the score for ":data/house".
                score /= 10000
            record[f"{part.value}_score"] = score
    return record


def _compute_ranks(dataset_df: pd.DataFrame) -> pd.DataFrame:
    dataset_df = dataset_df.sort_values(['test_mean', 'test_std'], ascending=[False, True])
    ranks = []
    current_score = None
    current_std = None
    for _, columns in dataset_df.iterrows():
        score = columns['test_mean']
        std = columns['test_std']
        if current_score is None:
            ranks.append(1)
            current_score = score
            current_std = std
        elif current_score - score <= current_std:
            ranks.append(ranks[-1])
        else:
            ranks.append(ranks[-1] + 1)
            current_score = score
            current_std = std
    dataset_df['rank'] = ranks
    return dataset_df


def build_metrics_dataframe(
    outputs_info: list[
        tuple[
            Union[str, Path],  # output path
            str,  # key (for example, algorithm name: "MLP")
            Union[int, str],  # subkey for aggregation (for example, seed: 0)
        ]
    ],
    precision: Optional[int] = 4,
):
    # >>> Build dataframe.
    records = [
        load_record(output) | { 'key': key, 'subkey': str(subkey)}
        for output, key, subkey in outputs_info
        if lib.get_path(output).joinpath('DONE').exists()
    ]
    if not records:
        raise RuntimeError('No records are available')
    df = pd.DataFrame.from_records(records)
    has_train_score = 'train_score' in df.columns

    # >>> Aggregate over subkeys.
    aggregations = {
        'test_mean': ("test_score", "mean"),
        'test_std': ("test_score", "std"),
        'val_mean': ("val_score", "mean"),
        'val_std': ("val_score", "std"),
    }
    if has_train_score:
        aggregations.update({
            'train_mean': ("train_score", "mean"),
            'train_std': ("train_score", "std"),
        })
    aggregations['count'] = ("test_score", "count")
    aggregations.update({
        x: (x, "first")
        for x in DATASET_PROPERTIES
        if x in df.columns
    })
    df = df.groupby(["dataset", "key"]).agg(**aggregations)
    df = df.reset_index().fillna(0.0)
    df["count"] = df["count"].astype(int)

    # >>> Compute ranks.
    df = cast(
        pd.DataFrame,
        df.groupby(['dataset'], group_keys=False).apply(_compute_ranks)
    )

    # >>> Finalize.
    df = df.sort_values(
        ['size', 'n_features', 'dataset', 'test_mean'],
        ascending=[True, True, True, False],
    ).reset_index(drop=True)
    df.loc[
        df['task_type'] == 'regression',
        ['test_mean', 'val_mean'] + ['train_mean'] * int(has_train_score)
    ] *= -1
    if precision is not None:
        float_columns = [
            'test_mean', 'test_std',
            'val_mean', 'val_std',
        ] + ['train_mean', 'train_std'] * int(has_train_score)
        df[float_columns] = df[float_columns].round(precision)
    df = df.set_index(["dataset"] + DATASET_PROPERTIES + ["key"])
    return df


def summarize_ranks(metrics_df: pd.DataFrame, nans: bool) -> pd.DataFrame:
    df = metrics_df
    df = df.reset_index().pivot(index='key', columns='dataset', values='rank')
    if not nans:
        df = df.dropna(axis='columns')
    columns = df.columns.tolist()
    df["avg"] = df.mean(1)
    df["std"] = df.std(1)
    df.insert(0, "avg", df.pop("avg").round(1))
    df.insert(1, "std", df.pop("std").round(1))
    df = df.sort_values("avg")
    df = df[['avg', 'std'] + columns]
    return df

# How to use the next cell
- comment/uncomment `N_SEEDS += 15` to show/hide results for single models
- comment/uncomment `N_ENSEMBLES += 3` to show/hide results for ensembles
- in the `for dataset in datasets` loop:
    - comment/uncomment the `add(...)` lines to show/hide results for various algorithms
    - in particular, uncomment `add(f':exp/mlp/{dataset}/0-reproduce', 'MLP (reproduce)')` to complete the tutorial from `README.md`

In [10]:
N_SEEDS = 1
N_ENSEMBLES = 1
N_ENSEMBLES += 15

# See the comments in build_metrics_dataframe to learn about outputs_info.
outputs_info = []
def add(location: str, name: Optional[str] = None, sep: str = '-'):
    if name is None:
        assert location.startswith(':exp/')
        # location example: ":exp/mlp/california/0"
        _exp_prefix, alg, *_dataset, tag = location.split('/')
        name = f'{alg}[{tag}]'
    for seed in range(N_SEEDS):
        outputs_info.append((location + f'{sep}evaluation/{seed}', name, seed))
    for ensemble_i in range(N_ENSEMBLES):
        outputs_info.append((location + f'{sep}ensemble{sep}5/{ensemble_i}', '(E) ' + name, ensemble_i))

datasets = DATASETS_MAIN
for dataset in datasets:
    if dataset in DATASETS_WHY:
        dataset = 'why/' + dataset

    # >>> Tutorial from README.md
    # add(f':exp/mlp/{dataset}/0-reproduce', 'MLP (reproduce)')

    # >>> Retrieval-augmented baselines
    # add(f':exp/knn/{dataset}/0', 'kNN')

    # dnnr_tag = 'ohe' if dataset in [BLACK_FRIDAY, DIAMOND] else 'loo'
    # add(f':exp/dnnr/{dataset}/{dnnr_tag}', 'DNNR')

    # add(f':exp/anp/{dataset}/0', 'ANP')
    # add(f':exp/dkl/{dataset}/0', 'DKL')

    # npt_tag = {
    #     'churn': 0,
    #     'california': 0,
    #     'house': 0,
    #     'adult': 0,
    #     'diamond': 2,
    #     'otto': 1,
    #     'higgs-small': 2,
    #     'black-friday': 2,
    #     'covtype': 3,
    #     'weather-small': 1,
    #     'microsoft': 1,
    # }[dataset]
    # add(f':exp/npt/{dataset}/{npt_tag}', 'NPT')

    # saint_tag = 'default' if dataset in ['weather-small', 'covtype', 'microsoft'] else '2'
    # add(f':exp/saint/{dataset}/{saint_tag}', 'SAINT')

    # >>> Parametric DL baselines
    add(f':exp/mlp/{dataset}/0', 'MLP')
    # add(f':exp/mlp/{dataset}/lr', 'MLP-LR')
    # add(f':exp/mlp/{dataset}/plr-lite', 'MLP-PLR(lite)')
    add(f':exp/mlp/{dataset}/plr', 'MLP-PLR')

    # >>> GBDT
    # add(f':exp/xgboost_/{dataset}/default2', 'XGBoost (default)')
    # add(f':exp/lightgbm_/{dataset}/default2', 'LightGBM (default)')
    # add(f':exp/catboost_/{dataset}/default2', 'CatBoost (default)')
    add(f':exp/xgboost_/{dataset}/2', 'XGBoost')
    add(f':exp/lightgbm_/{dataset}/2', 'LightGBM')
    add(f':exp/catboost_/{dataset}/2', 'CatBoost')

    # >>> The model
    model = 'TabR'
    modeldir = model.lower()
    # add(f':exp/{modeldir}/{dataset}/default', f'{model}-S (default)')
    # add(f':exp/{modeldir}/{dataset}/0', f'{model}-S')
    model_tag = "2-lr" if dataset in ['weather-small', 'covtype', 'microsoft'] else "2-plr-lite"
    add(f':exp/{modeldir}/{dataset}/{model_tag}', f'{model}')

    # >>> Ablation study
    for tag, name in [
        # ('dp-qk-v-self-scaled', 'Step-0'),
        # ('dp-qk-yv-self-scaled', 'Step-1'),
        # ('l2-k-yv-self-scaled', 'Step-2'),
        # ('l2-k-yt-self-scaled', 'Step-3'),
    ]:
        add(f':exp/{modeldir}_design/{dataset}/{tag}', f'(design) {name}')

    # >>> Context freeze
    for freeze_after_n_epochs in [
        # 0,
        # 1,
        # 2,
        # 4,
        # 5,
        # 8,
    ]:
        add(f':exp/{modeldir}_scaling/{dataset}/default-freeze-{freeze_after_n_epochs}', f'{model}-freeze-{freeze_after_n_epochs}')

metrics_df = build_metrics_dataframe(outputs_info)
# Drop details about datasets to save screen space.
# while len(metrics_df.index.levels) > 2:
#     metrics_df.index = metrics_df.index.droplevel(1)
ranks_df = summarize_ranks(metrics_df, nans=True)
print('Ranks:')
display(ranks_df)
print('\nMetrics:')
display(metrics_df)
# metrics_df.to_html('metrics.html')
# ranks_df.to_html('metrics.html')

Ranks:


dataset,avg,std,AD
key,,,
(E) CatBoost,1.0,0.0,1
(E) LightGBM,1.0,0.0,1
(E) MLP-PLR,1.0,0.0,1
(E) TabR,1.0,0.0,1
(E) XGBoost,1.0,0.0,1
CatBoost,1.0,0.0,1
XGBoost,1.0,0.0,1
LightGBM,2.0,0.0,2
TabR,3.0,0.0,3



Metrics:


test_mean  test_std  \
dataset task_type size  n_features key                                 
AD      binclass  48842 14         (E) MLP-PLR      0.8726    0.0008   
                                   (E) XGBoost      0.8723    0.0002   
                                   (E) CatBoost     0.8723    0.0007   
                                   (E) TabR         0.8722    0.0005   
                                   XGBoost          0.8722    0.0000   
                                   (E) LightGBM     0.8721    0.0004   
                                   CatBoost         0.8721    0.0000   
                                   LightGBM         0.8711    0.0000   
                                   TabR             0.8706    0.0000   
                                   MLP-PLR          0.8694    0.0000   
                                   (E) MLP          0.8540    0.0002   
                                   MLP              0.8539    0.0000   

                                                 val_mean  val_std  \
dataset task_type size  n_features key                               
AD      binclass  48842 14         (E) MLP-PLR     0.8719   0.0005   
                                   (E) XGBoost     0.8709   0.0012   
                                   (E) CatBoost    0.8724   0.0005   
                                   (E) TabR        0.8748   0.0012   
                                   XGBoost         0.8739   0.0000   
                                   (E) LightGBM    0.8729   0.0006   
                                   CatBoost        0.8750   0.0000   
                                   LightGBM        0.8753   0.0000   
                                   TabR            0.8756   0.0000   
                                   MLP-PLR         0.8736   0.0000   
                                   (E) MLP         0.8579   0.0014   
                                   MLP             0.8592   0.0000   

                                                 train_mean  train_std  count  \
dataset task_type size  n_features key                                          
AD      binclass  48842 14         (E) MLP-PLR       0.8849     0.0054      3   
                                   (E) XGBoost       0.9144     0.0042      3   
                                   (E) CatBoost      0.8976     0.0050      3   
                                   (E) TabR          0.8856     0.0007      3   
                                   XGBoost           0.9210     0.0000      1   
                                   (E) LightGBM      0.9065     0.0022      3   
                                   CatBoost          0.8968     0.0000      1   
                                   LightGBM          0.9057     0.0000      1   
                                   TabR              0.8807     0.0000      1   
                                   MLP-PLR           0.8968     0.0000      1   
                                   (E) MLP           0.8806     0.0024      3   
                                   MLP               0.8801     0.0000      1   

                                                 rank  
dataset task_type size  n_features key                 
AD      binclass  48842 14         (E) MLP-PLR      1  
                                   (E) XGBoost      1  
                                   (E) CatBoost     1  
                                   (E) TabR         1  
                                   XGBoost          1  
                                   (E) LightGBM     1  
                                   CatBoost         1  
                                   LightGBM         2  
                                   TabR             3  
                                   MLP-PLR          4  
                                   (E) MLP          5  
                                   MLP              5